# Week 12 - Generative models  

## Part 2: Variational Autoencoders (VAEs)



This part we move towards a non-linear mapping between the latent space z and the observation/input space x. Concretely, we will implement a VAE in Pyro.

Import required libraries:

In [1]:
# Install Pyro, if necessary
#!pip install pyro-ppl

In [2]:
import numpy as np
import torch
import torch.nn as nn
from mnist_cached import MNISTCached as MNIST
from mnist_cached import setup_data_loaders
from vae_plots import mnist_test_tsne, plot_llk, plot_vae_samples

import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam

# fix random generator seed (for reproducibility of results)
np.random.seed(42)

# matplotlib style options
from matplotlib import pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

/Users/work/Documents/model-based-machine-learning-ssvae/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
torch.manual_seed(1234)
np.random.seed(1234)

## Variational Autoencoders in Pyro

We will now implement a VAE in Pyro. Let us begin by recapping the model introduced in the class. The generative process can be summarize as follows:

1. For each observation $n \in \{1,\dots,N\}$
    1. Sample latent projection $\textbf{z}_n \sim \mathcal{N}(\textbf{z}_n|\textbf{0}, \textbf{I})$
    2. Sample observation $\textbf{x}_n \sim \mathcal{N}(\textbf{x}_n|f_{\boldsymbol\theta}(\textbf{z}_n))$

Note that the neural network $f$ is outputting both the mean and variance of the Gaussian. 

Our goal is to compute the posterior distribution over the latent variables $\textbf{z}_n$, while jointly finding point estimates for the parameters of that neural network $\boldsymbol\theta$. Note that you could also treat $\boldsymbol\theta$ as latent variables and perform inference on them as well (i.e. a Bayesian neural network), but, for this particular model, point estimates are usually sufficient. 

We will also need an approximate distribution $q(\textbf{z})$ for VI. As we saw in the slides, we will use a second neural network $g$ to parameterize this variational distribution $q$. Therefore, we can represent the entire process of the training a VAE as follows:

<img style="width:50%" src="http://mlsm.man.dtu.dk/vae.png">

In the diagram, $g$ is the encoder network and $f$ is the decoder network. Lets begin by implementing those two neural networks in Torch:

In [4]:
# define the PyTorch module that parameterizes the
# diagonal gaussian distribution q(z|x)
class Encoder(nn.Module):
    def __init__(self, z_dim, hidden_dim, input_dim):
        self.input_dim = input_dim
        super().__init__()
        # setup the three linear transformations used
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc21 = nn.Linear(hidden_dim, z_dim)
        self.fc22 = nn.Linear(hidden_dim, z_dim)
        # setup the non-linearities
        self.softplus = nn.Softplus()

    def forward(self, x):
        # define the forward computation on the image x
        # first shape the mini-batch to have pixels in the rightmost dimension
        x = x.reshape(-1, self.input_dim)
        # then compute the hidden units
        hidden = self.softplus(self.fc1(x))
        # then return a mean vector and a (positive) square root covariance
        # each of size batch_size x z_dim
        z_loc = self.fc21(hidden)
        z_scale = torch.exp(self.fc22(hidden))
        return z_loc, z_scale


# define the PyTorch module that parameterizes the
# observation likelihood p(x|z)
class Decoder(nn.Module):
    def __init__(self, z_dim, hidden_dim, output_dim, binary=False):
        self.binary = binary
        super().__init__()
        # setup the two linear transformations used
        self.fc1 = nn.Linear(z_dim, hidden_dim)
        self.fc21 = nn.Linear(hidden_dim, output_dim)
        if not self.binary:
            self.fc22 = nn.Linear(hidden_dim, output_dim)
        # setup the non-linearities
        self.softplus = nn.Softplus()

    def forward(self, z):
        # define the forward computation on the latent z
        # first compute the hidden units
        hidden = self.softplus(self.fc1(z))
        # return the parameter for the output Bernoulli
        # each is of size batch_size x input_dim
        if self.binary:
            out = torch.sigmoid(self.fc21(hidden))
        else:
            out = (self.fc21(hidden), self.softplus(self.fc22(hidden)))
        return out

Now that we have defined $f$ and $g$, we can use them to implement the model() and guide() in Pyro:

- The guide() will take the observations $\textbf{x}$ as input, pass them through the encoder network $g$, and use the output of the encoder to parameterize the variational approximation $q(\textbf{z})$. You can do a forward pass through the encoder using: ``self.encoder.forward(x)``. 

- The model() just follows the generative process of the VAE described above. For each observation $n$, it samples $\textbf{z}_n \sim \mathcal{N}(\textbf{z}_n|\textbf{0}, \textbf{I})$ and passes the samples through the decoder network $f$, in order to produce the parameters of the observation distributon $\mathcal{N}(\textbf{x}_n|f_{\boldsymbol\theta}(\textbf{z}_n))$. You can do a forward pass through the decoder using: ``self.decoder.forward(z)``. 

Can try to you implement the guide() and model() for the Pyro model below? 

In [5]:

# define a PyTorch module for the VAE
class VAE(nn.Module):
    # by default our latent space is 50-dimensional
    # and we use 400 hidden units
    def __init__(self, z_dim=50, hidden_dim=400, input_dim=784, binary=False, use_cuda=False):
        self.input_dim = input_dim
        self.binary = binary
        super().__init__()
        # create the encoder and decoder networks
        self.encoder = Encoder(z_dim, hidden_dim, input_dim)
        self.decoder = Decoder(z_dim, hidden_dim, input_dim, binary=binary)

        if use_cuda:
            # calling cuda() here will put all the parameters of
            # the encoder and decoder networks into gpu memory
            self.cuda()
        self.use_cuda = use_cuda
        self.z_dim = z_dim

    # define the model p(x|z)p(z)
    def model(self, x):
        # register PyTorch module `decoder` with Pyro
        pyro.module("decoder", self.decoder)

        with pyro.plate("data", x.shape[0]):
            z_loc = torch.zeros(x.shape[0], self.z_dim)
            z_scale = torch.ones(x.shape[0], self.z_dim)
            z = pyro.sample("z", dist.Normal(z_loc, z_scale).to_event(1))
            img = self.decoder.forward(z)
            if self.binary:
                # sample from the Bernoulli distribution
                pyro.sample("obs", dist.Bernoulli(img, validate_args=False).to_event(1), obs=x.reshape(-1, self.input_dim))
            else:
                # sample from the Gaussian distribution
                loc, scale = img
                pyro.sample("obs", dist.Normal(loc, scale, validate_args=False).to_event(1), obs=x.reshape(-1, self.input_dim))

    # define the guide (i.e. variational distribution) q(z|x)
    def guide(self, x):
        # register PyTorch module `encoder` with Pyro
        pyro.module("encoder", self.encoder)
        
        with pyro.plate("data", x.shape[0]):
            z_loc, z_scale = self.encoder.forward(x)
            z = pyro.sample("z", dist.Normal(z_loc, z_scale).to_event(1))


    # define a helper function for reconstructing images (this will useful later - ignore for now)
    def reconstruct_img(self, x):
        # encode image x
        z_loc, z_scale = self.encoder(x)
        # sample in latent space
        z = dist.Normal(z_loc, z_scale).sample()
        # decode the image (note we don't sample in image space)
        loc_img = self.decoder(z)
        return loc_img

Now that the Pyro model is defined, lets run Bayesian inference on it. Note that the encoder and decoder networks will be trained jointly as part of the VI process.

In [6]:
# clear param store
# pyro.clear_param_store()

# setup the VAE
# vae = VAE(z_dim=2, hidden_dim=20, input_dim=4, use_cuda=False)

# setup the optimizer
# adam_args = {"lr": 1.0e-3}
# optimizer = Adam(adam_args)

# setup the inference algorithm
#elbo = Trace_ELBO()
# elbo = Trace_ELBO(num_particles=3)
# svi = SVI(vae.model, vae.guide, optimizer, loss=elbo)

# train_elbo = {}
# test_elbo = {}
# training loop
# num_epochs = 5000
# for epoch in range(num_epochs):
#     # do ELBO gradient and accumulate loss
#     epoch_loss = svi.step(X)

#     if epoch % 100 == 0:
#         # report training diagnostics
#         total_epoch_loss_train = epoch_loss / len(X)
#         train_elbo[epoch] = total_epoch_loss_train
#         print("[epoch %03d]  average training loss: %.4f" % (epoch, total_epoch_loss_train))

As we did for PPCA, we can look at the reconstructions of the inputs $\textbf{x}$ and see how good they are.

## VAE application in computer vision

The Iris data is relatively simple. Lets now look at a more complex application in computer vision, where the VAE can really start to shine. Concretely, we will try to model images of digits using a VAE - the MNIST dataset. In the code below, we will load the datasets and train the VAE model on it. We will use a larger dimensionality (``z_dim=50``) of the latent space $\textbf{z}$.

Note: training the VAE on this dataset will take a few minutes (about 5-10 minutes, depending on your computer).

In [7]:
use_cuda = False

# clear param store
pyro.clear_param_store()




In [ ]:
loaders = setup_data_loaders(MNIST, use_cuda=False, batch_size=1, sup_num=49000)
train_loader = loaders["sup"]
test_loader = loaders["test"]

In [11]:


# setup the VAE
vae = VAE(use_cuda=use_cuda, z_dim=50, hidden_dim=400, input_dim=784, binary=True)

# setup the optimizer
adam_args = {"lr": 1.0e-3}
optimizer = Adam(adam_args)

# setup the inference algorithm
elbo = Trace_ELBO()
svi = SVI(vae.model, vae.guide, optimizer, loss=elbo)

train_elbo = {}
test_elbo = {}
# training loop
num_epochs = 30
for epoch in range(num_epochs):
    # initialize loss accumulator
    epoch_loss = 0.0
    # do a training epoch over each mini-batch x returned
    # by the data loader
    for x, _ in train_loader:
        # if on GPU put mini-batch into CUDA memory
        print(x.shape)
        if use_cuda:
            x = x.cuda()
        # do ELBO gradient and accumulate loss
        epoch_loss += svi.step(x)

    if epoch % 5 == 0:
        # report training diagnostics
        normalizer_train = len(train_loader.dataset)
        total_epoch_loss_train = epoch_loss / normalizer_train
        train_elbo[epoch] = total_epoch_loss_train
        print("[epoch %03d]  average training loss: %.4f" % (epoch, total_epoch_loss_train))
        
        # initialize loss accumulator
        test_loss = 0.0
        # compute the loss over the entire test set
        for i, (x, _) in enumerate(test_loader):
            # if on GPU put mini-batch into CUDA memory
            if use_cuda:
                x = x.cuda()
            # compute ELBO estimate and accumulate loss
            test_loss += svi.evaluate_loss(x)

        # report test diagnostics
        normalizer_test = len(test_loader.dataset)
    
        total_epoch_loss_test = test_loss / normalizer_test
        test_elbo[epoch] = total_epoch_loss_test
        print("[epoch %03d]  average test loss: %.4f" % (epoch, total_epoch_loss_test))
        plot_llk(train_elbo, test_elbo)

torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1, 784])
torch.Size([1

KeyboardInterrupt: 

We can now look at the reconstructions produced by the VAE model for a subset of the digits in the dataset:

In [133]:
# pick 10 random test images from the first mini-batch and visualize how well we're reconstructing them
fig, axs = plt.subplots(10,2, figsize=(6,20))
x = next(iter(test_loader))[0]
# if on GPU put mini-batch into CUDA memory
if use_cuda:
    x = x.cuda()
#plot_vae_samples(vae, vis)
reco_indices = np.random.randint(0, x.shape[0], 10)
for i in range(10):
    index = reco_indices[i]
    test_img = x[index, :]
    reco_img = vae.reconstruct_img(test_img)
    #vis.image(
    #    test_img.reshape(28, 28).detach().cpu().numpy(),
    #    opts={"caption": "test image"},
    #)
    #vis.image(
    #    reco_img.reshape(28, 28).detach().cpu().numpy(),
    #    opts={"caption": "reconstructed image"},
    #)
    axs[i,0].imshow(test_img.reshape(28, 28).detach().cpu().numpy(), cmap="Greys")
    axs[i,1].imshow(reco_img.reshape(28, 28).detach().cpu().numpy(), cmap="Greys")
    
axs[0,0].set_title("True image");
axs[0,1].set_title("Reconstruction");

In [134]:
# Grab the first batch from test_loader
x_batch, y_batch = next(iter(test_loader))

# Print the first 10 labels
print(y_batch[:10])

tensor([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]])


We can also try to visualize the projections of the digits in the latent space. However, in this case, the latent projections are still 50-dimensional!! We will need to project them down even further if we want to visualize them on a 2-D plot. For this, we will use a dimensionality reduction called t-SNE. t-SNE tries to project down the data while attempting to perserve the (Euclidean) distances between all pairs of data points (observations) in the dataset. Lets do that and see how it looks:

In [135]:
mnist_test_tsne(vae=vae, test_loader=test_loader)

/Users/work/Documents/model-based-machine-learning-ssvae/.venv/lib/python3.9/site-packages/torchvision/datasets/mnist.py:81: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/Users/work/Documents/model-based-machine-learning-ssvae/.venv/lib/python3.9/site-packages/torchvision/datasets/mnist.py:71: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
/Users/work/Documents/model-based-machine-learning-ssvae/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:335: RuntimeWarning: divide by zero encountered in matmul
  Q, _ = normalizer(A @ Q)
/Users/work/Documents/model-based-machine-learning-ssvae/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:335: RuntimeWarning: overflow encountered in matmul
  Q, _ = normalizer(A @ Q)
/Users/work/Documents/model-based-machine-learning-ssvae/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:335: RuntimeWarning: invalid value e

As we can see, the observations of the different digits are nicely separated (clustered) in the latent space $\textbf{z}$. I.e., all observations of the digit 5 are clustered together in the latent space, and similarly for other digits.

Lastly, since the VAE is a generative model, we can use it to sample new observations ("fake data"). We can do this by sampling $z$ from a standard Gaussian distribution and passing those samples through the decoder network. 

Lets do this to generate 10 new samples of digit images:

In [136]:
plt.figure()
for i in range(10):
    z_loc = torch.zeros(vae.z_dim, dtype=x.dtype, device=x.device)
    z_scale = torch.ones(vae.z_dim, dtype=x.dtype, device=x.device)
    z = dist.Normal(z_loc, z_scale).rsample()
    out = vae.decoder.forward(z)
    # plt.figure(figsize=(3,3))
    plt.imshow(out.reshape(28, 28).detach().cpu().numpy(), cmap="Greys")


------------------------------------------------------------------------

--------------------------------------------------------------------------------
2) Define a Pyro model and guide for a multiclass linear classifier
--------------------------------------------------------------------------------

We will create a K-class linear model y ∼ Categorical(logits = z_loc @ W + b).  

Note: for Iris, K=3 classes, but this approach generalizes to any K ≥ 2.

------------------------------------------------------------------------

In [137]:

num_classes = 10 # as we have 10 digits in MNIST


In [199]:
def classifier_model(x, obs=None):
    z = vae.encoder(x)[0]
    alpha = pyro.sample("alpha_c", pyro.distributions.Normal(0., 1.).expand( [num_classes]).to_event())
    beta = pyro.sample("beta_c", pyro.distributions.Normal(0., 1.).expand([z.shape[1], num_classes]).to_event())
    logits =   z @ beta + alpha
    with pyro.plate("data", z.shape[0]):
        y = pyro.sample("y", pyro.distributions.Categorical(logits=logits), obs=obs)


------------------------------------------------------------------------

--------------------------------------------------------------------------------
3) Train the classifier in Pyro using SVI
--------------------------------------------------------------------------------

You can run standard Pyro SVI over these model/guide definitions. For example:

------------------------------------------------------------------------

In [139]:
from pyro.contrib.autoguide import AutoDiagonalNormal, AutoMultivariateNormal


In [ ]:
# Clear out any existing parameter values
# pyro.clear_param_store()

classifier_guide = AutoMultivariateNormal(classifier_model)

optimizer = Adam({"lr": 1.0e-3})
# Setup SVI
svi_classifier = SVI(
    model=classifier_model,
    guide=classifier_guide,
    optim=optimizer,
    loss=Trace_ELBO(num_particles=3),
)
num_steps = 100  # Adjust as needed
for step in range(num_steps):
    epoch_loss = 0.0
    for x, y in train_loader:
        # if on GPU put mini-batch into CUDA memory
        if use_cuda:
            x = x.cuda()
            y = y.cuda()
        y = np.argmax(y, axis=1) # FIX THIS. Ensure that it is converted in the right way
        # do ELBO gradient and accumulate loss
        epoch_loss += svi_classifier.step(x, y)
    if step % 5 == 0:
        print(f"[step {step}] loss = {epoch_loss:.2f}")


[step 0] loss = 306309.40
[step 5] loss = 144272.46
[step 10] loss = 95709.15
[step 15] loss = 78196.79
[step 20] loss = 73698.48
[step 25] loss = 72477.46
[step 30] loss = 71854.16
[step 35] loss = 71674.86
[step 40] loss = 71638.49
[step 45] loss = 71733.55
[step 50] loss = 71770.13
[step 55] loss = 71689.99
[step 60] loss = 71853.11
[step 65] loss = 71610.06
[step 70] loss = 71439.43
[step 75] loss = 71492.18
[step 80] loss = 71522.79
[step 85] loss = 71559.27
[step 90] loss = 71533.05
[step 95] loss = 71467.81


In [ ]:
# pick 10 random test images from the first mini-batch and visualize how well we're reconstructing them
fig, axs = plt.subplots(10,2, figsize=(6,20))
x = next(iter(train_loader))[0]
# if on GPU put mini-batch into CUDA memory
if use_cuda:
    x = x.cuda()
#plot_vae_samples(vae, vis)
reco_indices = np.random.randint(0, x.shape[0], 10)
for i in range(10):
    index = reco_indices[i]
    test_img = x[index, :]
    reco_img = vae.reconstruct_img(test_img)
    axs[i,0].imshow(test_img.reshape(28, 28).detach().cpu().numpy(), cmap="Greys")
    axs[i,1].imshow(reco_img.reshape(28, 28).detach().cpu().numpy(), cmap="Greys")
    
axs[0,0].set_title("True image");
axs[0,1].set_title("Reconstruction");


# Get true labels
true_labels = torch.argmax(y_batch[reco_indices], dim=1)

for i in range(10):
    index = reco_indices[i]
    axs[i,0].set_xlabel(f"True: {true_labels[i].item()}")
    # axs[i,1].set_xlabel(f"Predicted: {predicted_labels[i].item()}")

In [142]:
plt.imshow(reco_img.reshape(28, 28).detach().cpu().numpy())

In [143]:
%matplotlib inline


In [212]:

# define a PyTorch module for the VAE
class ssVAE(nn.Module):
    # by default our latent space is 50-dimensional
    # and we use 400 hidden units
    def __init__(self, z_dim=50, hidden_dim=400, input_dim=784, binary=False, use_cuda=False):
        self.input_dim = input_dim
        self.binary = binary
        super().__init__()
        # create the encoder and decoder networks
        self.encoder = Encoder(z_dim, hidden_dim, input_dim)
        self.decoder = Decoder(z_dim, hidden_dim, input_dim, binary=binary)

        if use_cuda:
            # calling cuda() here will put all the parameters of
            # the encoder and decoder networks into gpu memory
            self.cuda()
        self.use_cuda = use_cuda
        self.z_dim = z_dim

    # define the model p(x|z)p(z)
    def model(self, x, y=None):
        BATCH_SIZE = x.shape[0]
        # register PyTorch module `decoder` with Pyro
        pyro.module("decoder", self.decoder)
        alpha = pyro.sample("alpha", pyro.distributions.Normal(0., 1.).expand([num_classes]).to_event(1))
        beta = pyro.sample("beta", pyro.distributions.Normal(0., 1.).expand([self.z_dim, num_classes]).to_event(2))
        with pyro.plate("data", x.shape[0]):
            z_loc = torch.zeros(x.shape[0], self.z_dim)
            z_scale = torch.ones(x.shape[0], self.z_dim)
            z = pyro.sample("z", dist.Normal(z_loc, z_scale).to_event(1))
            # Prepare for classification
            logits =   z @ beta + alpha
            # Reconstruct image
            img = self.decoder.forward(z)

            # Sample from the observation likelihood
            if self.binary:
                # sample from the Bernoulli distribution
                pyro.sample("obs", dist.Bernoulli(img, validate_args=False).to_event(1), obs=x.reshape(-1, self.input_dim))
                pyro.sample("y", dist.Categorical(logits=logits), obs=y)

            else:
                # sample from the Gaussian distribution
                loc, scale = img
                pyro.sample("obs", dist.Normal(loc, scale, validate_args=False).to_event(1), obs=x.reshape(-1, self.input_dim))
                pyro.sample("y", dist.Categorical(logits=logits), obs=y)

    def guide(self, x, y=None):
        # register PyTorch modules with Pyro
        pyro.module("encoder", self.encoder)

        # approximate posterior over alpha and beta
        alpha_loc = pyro.param("alpha_loc", torch.zeros(num_classes))
        alpha_scale = pyro.param("alpha_scale", torch.ones(num_classes))
        pyro.sample("alpha", dist.Normal(alpha_loc, alpha_scale).to_event(1))

        beta_loc = pyro.param("beta_loc", torch.zeros(self.z_dim, num_classes))
        beta_scale = pyro.param("beta_scale", torch.ones(self.z_dim, num_classes))
        beta = pyro.sample("beta", dist.Normal(beta_loc, beta_scale).to_event(2))
        
        with pyro.plate("data", x.shape[0]):
            # approximate posterior over z
            z_loc, z_scale = self.encoder.forward(x)
            z = pyro.sample("z", dist.Normal(z_loc, z_scale).to_event(1))

            
            # for unlabeled data, sample y from the classifier
            if y is None:
                logits = z @ beta + alpha_loc
                pyro.sample("y", dist.Categorical(logits=logits))
                
    def classifier(self, x):
        z_loc, z_scale = self.encoder(x)
        z = z_loc
        alpha_loc = pyro.param("alpha_loc")
        beta_loc = pyro.param("beta_loc")
        logits = z @ beta_loc + alpha_loc
        return torch.argmax(logits, dim=1)


    # define a helper function for reconstructing images (this will useful later - ignore for now)
    def reconstruct_img(self, x):
        # encode image x
        z_loc, z_scale = self.encoder(x)
        # sample in latent space
        z = dist.Normal(z_loc, z_scale).sample()
        # decode the image (note we don't sample in image space)
        loc_img = self.decoder(z)
        return loc_img

In [215]:

pyro.clear_param_store()

# setup the VAE
ssvae = ssVAE(use_cuda=use_cuda, z_dim=50, hidden_dim=400, input_dim=784, binary=True)

# setup the optimizer
adam_args = {"lr": 1.0e-3}
optimizer = Adam(adam_args)

# setup the inference algorithm
elbo = Trace_ELBO()
svi = SVI(ssvae.model, ssvae.guide, optimizer, loss=elbo)

train_elbo = {}
test_elbo = {}
# training loop
num_epochs = 30
for epoch in range(num_epochs):
    # initialize loss accumulator
    epoch_loss = 0.0
    # do a training epoch over each mini-batch x returned
    # by the data loader
    for x, y in train_loader:
        # if on GPU put mini-batch into CUDA memory
        y = np.argmax(y, axis=1)
        if use_cuda:
            x = x.cuda()
            y = y.cuda()
        # do ELBO gradient and accumulate loss
        epoch_loss += svi.step(x, y)

    if epoch % 5 == 0:
        # report training diagnostics
        normalizer_train = len(train_loader.dataset)
        total_epoch_loss_train = epoch_loss / normalizer_train
        train_elbo[epoch] = total_epoch_loss_train
        print("[epoch %03d]  average training loss: %.4f" % (epoch, total_epoch_loss_train))
        
        # initialize loss accumulator
        test_loss = 0.0
        # compute the loss over the entire test set
        for i, (x, y) in enumerate(test_loader):
            y = np.argmax(y, axis=1)
            # if on GPU put mini-batch into CUDA memory
            if use_cuda:
                x = x.cuda()
                y = y.cuda()
            
            # compute ELBO estimate and accumulate loss
            test_loss += svi.evaluate_loss(x, y)

        # report test diagnostics
        normalizer_test = len(test_loader.dataset)
    
        total_epoch_loss_test = test_loss / normalizer_test
        test_elbo[epoch] = total_epoch_loss_test
        print("[epoch %03d]  average test loss: %.4f" % (epoch, total_epoch_loss_test))
        plot_llk(train_elbo, test_elbo)

[epoch 000]  average training loss: 212.8823
[epoch 000]  average test loss: 175.1547
[epoch 005]  average training loss: 125.7275
[epoch 005]  average test loss: 121.7064
[epoch 010]  average training loss: 114.8575
[epoch 010]  average test loss: 112.8969
[epoch 015]  average training loss: 111.3463
[epoch 015]  average test loss: 109.7746
[epoch 020]  average training loss: 109.5555
[epoch 020]  average test loss: 108.4836
[epoch 025]  average training loss: 108.5502
[epoch 025]  average test loss: 107.3162


In [216]:
ssvae.classifier(test_img)

tensor([1])

In [207]:
np.argmax(y_batch[index])

tensor(6)

In [218]:
# Obtain both data (x_batch) and labels (y_batch) from the test loader
x_batch, y_batch = next(iter(test_loader))
if use_cuda:
    x_batch = x_batch.cuda()

fig, axs = plt.subplots(10, 4, figsize=(9, 20))

# Pick 10 random images to reconstruct
reco_indices = np.random.randint(0, x_batch.shape[0], 10)
for i in range(10):
    index = reco_indices[i]
    test_img = x_batch[index]

    # Reconstructions from ssVAE and VAE
    reco_img_ssvae = ssvae.reconstruct_img(test_img)
    reco_img_vae = vae.reconstruct_img(test_img)
    
    # Classifier returns a distribution, so extract the predicted label
    pred_label = int(ssvae.classifier(test_img))

    # True label
    true_label = np.argmax(y_batch[index])

    axs[i, 0].imshow(test_img.reshape(28, 28).detach().cpu().numpy(), cmap="Greys")
    axs[i, 1].imshow(reco_img_ssvae.reshape(28, 28).detach().cpu().numpy(), cmap="Greys")
    axs[i, 2].imshow(reco_img_vae.reshape(28, 28).detach().cpu().numpy(), cmap="Greys")

    # Fourth subplot: display predicted and true label
    axs[i, 3].axis("off")
    axs[i, 3].text(
        0.5,
        0.5,
        f"Pred: {pred_label}, True: {true_label}",
        fontsize=14,
        ha="center",
        va="center",
    )

axs[0, 0].set_title("True")
axs[0, 1].set_title("ssVAE")
axs[0, 2].set_title("VAE")
axs[0, 3].set_title("Pred/True Label")


Text(0.5, 1.0, 'Pred/True Label')

Things we are missing,

1. Trying to reload data with another fraction of Unsupervised and supervised.
2. Prepare a loss function for supervised and unsupervised. Making batches that have the same proportion of supervised datapoints. 
3. Reconstruction accuracy
4. Classifcation Accuracy
5. Logits entropy based AL
6. BALD (Bayesian Active Learning by Disagreement)
7. Documentation
8. Consider if should compare the T-SNE during our iterations (VAE, Supervised VAE, ssVAE)
9. Show what the samples we pick look like, and if they are actually just sample with high VAE reconstruction error
10. We might even be able to show sample location of the T-SNE plot.


In [ ]:
# First define data as Batchsize 1, then do manual batching afterwards.
# Hereby, it will be much easier to evaluate.

In [ ]:
def ss_loader(y, ss_fraction):
    shape = y.shape
    N = len(y)
    L1 = int(N * (1-ss_fraction))
    choices = np.random.choice(np.arange(N), L1, replace=False)
    y = np.array(y)
    y[choices] = -1
    return torch.tensor(y, dtype=torch.float32).reshape(shape)



In [ ]:
for x,y in train_loader:
    y = np.argmax(y, axis=1)
    print(y.shape)
    y = ss_loader(y, 0.1)
    print(sum(y==-1))
    break